In [9]:
# loading the API keys in environment variables
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

In [14]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

with open('actresses.txt') as f:
    actresses = f.read()


text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    length_function=len
)

In [18]:
chunks = text_splitter.create_documents([actresses])
print(chunks[2])
print(chunks[12].page_content)
print(f'This file is made up of  {len(chunks)} chunks')

page_content='generation, Streep has mesmerized audiences with her unparalleled talent and versatility for over'
to fully embody characters, coupled with her dedication to her craft, has made her a beacon of
This file is made up of  51 chunks


In [22]:
def calculate_cost(text):
    import tiktoken
    enc = tiktoken.encoding_for_model('text-embedding-ada-002')
    total_tokens = sum([len(enc.encode(page.page_content)) for page in text])
    print(f'Total Tokens: {total_tokens}')
    print(f'This costs in total in GBP: {total_tokens / 1000 * 0.0004:.6f}')
    
calculate_cost(chunks)

Total Tokens: 956
This costs in total in GBP: 0.000382


In [29]:
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

vector = embeddings.embed_query('cats')
vector

vector2 = embeddings.embed_query('dogs')
vector2

vector3 = embeddings.embed_query('rabbits')
vector3

[-0.017301304278350867,
 -0.012475287550756112,
 -0.012591577144119452,
 -0.03553292530360845,
 -0.018244542194667136,
 0.007242256315067093,
 -0.012927524754799927,
 -0.018502963720212907,
 -0.00407982571345149,
 -0.024782599899188126,
 0.013224708857251763,
 0.015259776275448928,
 -0.013147182772117062,
 -0.03297455611226012,
 0.01665525139580901,
 0.00770095410381568,
 0.024511259066877955,
 0.012507589542957403,
 0.02500225791650526,
 -0.015492355462175608,
 -0.0046031288835865205,
 0.005956610669026865,
 -0.008960757411036815,
 -0.0205315705537053,
 0.002172353911851018,
 -0.010446679785941146,
 0.0011943908184078827,
 -0.022353440539290104,
 -0.01438114420618039,
 -0.03067460658381241,
 0.02026022785874998,
 0.0006189997079595677,
 -0.02622975969718639,
 -0.013321616696500784,
 -0.011932602160845475,
 -0.01648727805613006,
 0.020828755587479708,
 0.004880931697585325,
 0.021319756299752163,
 0.01975630597706798,
 0.017236698431303137,
 0.012236247779324659,
 -0.003491917627591303

NameError: name 'embeddings' is not defined